In [1]:
# Hack to allow imports from directory above. Once the package is fully 
# functional, remove this and instead just install and import the package
import sys
sys.path.append("..")

import pandas as pd
from portfolio_manager.portfolio import InvestmentPortfolio
from portfolio_manager.return_calculators import StandardReturnCalculator, TimeWeightedReturnCalculator

In [2]:
# Create a portfolio
portfolio = InvestmentPortfolio(name='T212')

In [3]:
# Make some deposits
portfolio.deposit(10)
portfolio.deposit(20)

In [4]:
# Update the current portfolio value
portfolio.update_portfolio_value(75)

In [5]:
# Withdraw some funds
portfolio.withdraw(12.50)

In [6]:
# View the portfolio history
portfolio.portfolio_history

[{'date': datetime.datetime(2021, 1, 9, 13, 50, 53, 373977),
  'total_deposited': 10,
  'current_portfolio_value': 10,
  'transaction_type': 'deposit'},
 {'date': datetime.datetime(2021, 1, 9, 13, 50, 53, 374047),
  'total_deposited': 30,
  'current_portfolio_value': 30,
  'transaction_type': 'deposit'},
 {'date': datetime.datetime(2021, 1, 9, 13, 50, 53, 474463),
  'total_deposited': 30,
  'current_portfolio_value': 75,
  'transaction_type': 'update_portfolio_value'},
 {'date': datetime.datetime(2021, 1, 9, 13, 50, 53, 555545),
  'total_deposited': 17.5,
  'current_portfolio_value': 62.5,
  'transaction_type': 'withdrawal'}]

In [7]:
# View the portfolio history as a dataframe
df = pd.DataFrame(portfolio.portfolio_history, columns=['date', 'total_deposited', 'current_portfolio_value', 'transaction_type'])
df

,date,total_deposited,current_portfolio_value,transaction_type
0,2021-01-09 13:50:53.373977,10.0,10.0,deposit
1,2021-01-09 13:50:53.374047,30.0,30.0,deposit
2,2021-01-09 13:50:53.474463,30.0,75.0,update_portfolio_value
3,2021-01-09 13:50:53.555545,17.5,62.5,withdrawal


In [8]:
standard_return_calculator = StandardReturnCalculator()
portfolio_return = standard_return_calculator.calculate_return(portfolio, annualised=False)

In [9]:
portfolio_return = standard_return_calculator.calculate_return(portfolio, annualised=False)
portfolio_return

257.14285714285717

In [10]:
portfolio_return = standard_return_calculator.calculate_return(portfolio, annualised=True)
portfolio_return

52.85535436643993

In [11]:
portfolio.save_portfolio()

In [12]:
p = InvestmentPortfolio.load_portfolio('T212.pkl')

In [13]:
p.total_deposited

17.5

In [14]:
time_weighted_return_calculator = TimeWeightedReturnCalculator()
portfolio_return = time_weighted_return_calculator.calculate_return(portfolio, annualised=True)
portfolio_return

249.99999999999991